In [1]:
class DisjointSet:
    def __init__(self, vnum):
        self.parent=[-1 for _ in range(vnum)]

    def simple_find(self, i):
        while self.parent[i] >= 0:
            i=self.parent[i]
        return i

    def simple_union(self, i, j):
        self.parent[i]=j

    def collapsing_find(self, i):
        root=trail=lead=None
        # 루트를 찾는다.
        root=i
        while self.parent[root] >= 0:
            root=self.parent[root]

        # 모든 노드를 루트의 자식으로 만든다
        trail=i
        while trail != root:
            lead=self.parent[trail]
            self.parent[trail]=root
            trail=lead

        return root

    def weighted_union(self, i, j):
        """
        paremeters i, j must be roots!
        if size[i] < size[j] then parent[i]=j
        """
        #abs(parent[i])=size[i]
        #temp_cnt는 음수이고 size[i] + size[j]
        temp_cnt=self.parent[i]+self.parent[j]

        #size[i] < size[j]
        if self.parent[i] > self.parent[j]:
            self.parent[i]=j
            self.parent[j]=temp_cnt
        #size[i] > size[j]
        else:
            self.parent[j]=i
            self.parent[i]=temp_cnt

class Edge:
    def __init__(self, u, v, w):
        self.u=u
        self.v=v
        self.w=w

class Graph:
    def __init__(self, vertex_num):
        self.adj_list=[[] for _ in range(vertex_num)]
        self.edge_list=[]

        self.vertex_num=vertex_num

    def add_edge(self, u, v, weight):
        self.adj_list[u].append(v)
        self.adj_list[v].append(u)

        self.edge_list.append(Edge(u, v, weight))

    def MST_kruskal(self):
        mst=Graph(self.vertex_num)        
        ds=DisjointSet(self.vertex_num)
        self.edge_list.sort(key=lambda e: e.w)
        mst_edge_num=0
        edge_idx=0

        while mst_edge_num < self.vertex_num-1:
            edge=self.edge_list[edge_idx]
            if ds.collapsing_find(edge.u) != ds.collapsing_find(edge.v):
                mst.add_edge(edge.u, edge.v, edge.w)
                ds.weighted_union(ds.collapsing_find(edge.u), ds.collapsing_find(edge.v))
                mst_edge_num+=1
            edge_idx+=1

        return mst

    def print_edges(self):
        for edge in self.edge_list:
            print("({}, {}) : {}".format(edge.u, edge.v, edge.w))

In [1]:
class Element:
    def __init__(self, v, w, _from):
        # 가중치를 키로 사용한다
        self.w=w
        self.v=v
        self._from=_from

class MinHeap:
    MAX_ELEMENTS=200
    def __init__(self):
        self.arr=[None for i in range(self.MAX_ELEMENTS)]
        self.heapsize=0
        #정점이 arr에 위치한 현재 인덱스
        self.pos=[None for i in range(self.MAX_ELEMENTS)]

    def is_empty(self):
        if self.heapsize==0:
            return True
        return False

    def is_full(self):
        if self.heapsize>=self.MAX_ELEMENTS:
            return True
        return False

    def parent(self, idx):
        return idx >> 1

    def left(self, idx):
        return idx << 1

    def right(self, idx):
        return (idx << 1) + 1

    def push(self, item):
        if self.is_full():
            raise IndexError("the heap is full!!")

        self.heapsize+=1
        cur_idx=self.heapsize

        while cur_idx!=1 and item.w < self.arr[self.parent(cur_idx)].w: 
            self.arr[cur_idx]=self.arr[self.parent(cur_idx)]
            # pos의 인덱스는 정점, arr는 weight를 키로 만든 최소 힙
            self.pos[self.arr[cur_idx].v]=cur_idx

            cur_idx=self.parent(cur_idx)

        self.arr[cur_idx]=item
        self.pos[item.v]=cur_idx

    def pop(self):
        if self.is_empty():
            return None

        rem_elem=self.arr[1]

        temp=self.arr[self.heapsize]
        self.heapsize-=1

        cur_idx=1
        child=self.left(cur_idx)

        while child <= self.heapsize:
            if child < self.heapsize and \
                self.arr[self.left(cur_idx)].w > self.arr[self.right(cur_idx)].w:
                child=self.right(cur_idx)
            
            if temp.w <= self.arr[child].w:
                break

            self.arr[cur_idx]=self.arr[child]
            self.pos[self.arr[cur_idx].v]=cur_idx

            cur_idx=child
            child=self.left(cur_idx)
        
        self.arr[cur_idx]=temp
        self.pos[temp.v]=cur_idx

        return rem_elem

    def decrease_weight(self, new_elem):
        cur=self.pos[new_elem.v]

        while cur!= 1 and new_elem.w < self.arr[self.parent(cur)].w:
            self.arr[cur]=self.arr[self.parent(cur)]
            self.pos[self.arr[cur].v]=cur    

            cur=self.parent(cur)

        self.arr[cur]=new_elem
        self.pos[new_elem.v]=cur